## NLP task

### Import needed packages 

In [1]:
from transformers import pipeline
import pandas as pd
from tqdm import tqdm
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import os
import subprocess
import numpy as np
import matplotlib.pyplot as plt
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from transformers import BartForConditionalGeneration, BartTokenizer
import torch
warnings.filterwarnings("ignore")

### Read the file via pandas

In [2]:
excel_file_path = f'C:/Users/sdany/Desktop/pilot project/pilotprtfinal.xlsx'
df = pd.read_excel(excel_file_path)

In [3]:
df.describe()

,Solicitation Year,Award Year,Award Amount,DUNS,Number Employees
count,1615.000000,1615.000000,1.615000e+03,1.593000e+03,1605.000000
mean,2018.022910,2019.427864,5.847700e+05,2.596446e+08,50.300312
std,7.252831,2.684268,1.555689e+06,3.155795e+08,76.233102
min,1900.000000,2014.000000,2.490000e+04,0.000000e+00,0.000000
25%,2016.000000,2017.000000,1.398605e+05,7.943749e+07,6.000000
50%,2019.000000,2020.000000,1.500000e+05,1.169762e+08,17.000000
75%,2021.000000,2022.000000,7.499998e+05,1.884658e+08,57.000000
max,2023.000000,2023.000000,5.580000e+07,9.698515e+08,468.000000


In [4]:
## Labels and preprocess the text 

In [5]:
def change_labels(labels):
    label_map = {
        'Propulsion, motor, thruster, propplant': 'propulsion', 
        'Guidance, navigation, control, Attitude Determination': 'ACDS & GNC', 
        'System & Integration': 'System & Integration', 
        'Software': 'Software', 
        'Telecommunications': 'Communications', 
        'Sensors': 'Sensors',
        'Structure, Material & Mechanics': 'Mechanics',  
        'Command and Data Handling': 'C & DH',  
        'Power, energy': 'Power', 
        'Telemetry, Tracking and Command': 'TT & C'  
    }
    return label_map.get(labels, labels)

candidate_labels = [
    'Propulsion, motor, thruster, propplant', 
    'Guidance, navigation, control, Attitude Determination',
    'System & Integration',
    'Software', 
    'Telecommunications', 
    'Sensors',
    'Structure, Material & Mechanics',
    'Command and Data Handling', 
    'Power, energy', 
    'Telemetry, Tracking and Command'
]

# Function to preprocess text
def preprocess_text(text):
    if not isinstance(text, str):
        text = str(text)
    text = text.lower()
    return text

df['Abstract'] = df['Abstract'].apply(preprocess_text)

#### Model using and defineing tasks : two model for two task : Summa.. & Classi...

In [6]:
task = "zero-shot-classification"
modelId = "facebook/bart-large-mnli"
classifier = pipeline(task=task, model=modelId)
#summarizer = pipeline(task="summarization", model="sshleifer/distilbart-cnn-12-6")

### Process part 

In [7]:
results_summarization = []
results_classification = []
predicted_labels = []

#### Summarization

In [8]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from tqdm import tqdm

# Download necessary NLTK data
nltk.download('stopwords')
nltk.download('punkt')

# Initialize stopwords set
stopWords = set(stopwords.words("english"))


# Iterate through each abstract in the DataFrame
for abstract in tqdm(df['Abstract'], desc="Summarizing"):
    if len(word_tokenize(abstract)) > 512:
        # Tokenize words and create frequency table
        words = word_tokenize(abstract)
        freqTable = dict()
        for word in words:
            word = word.lower()
            if word not in stopWords:
                freqTable[word] = freqTable.get(word, 0) + 1
        
        # Tokenize sentences
        sentences = sent_tokenize(abstract)
        
        # Calculate sentence values
        sentenceValue = dict()
        for sentence in sentences:
            for word, freq in freqTable.items():
                if word in sentence.lower():
                    sentenceValue[sentence] = sentenceValue.get(sentence, 0) + freq
        
        # Calculate average sentence value
        sumValues = sum(sentenceValue.values())
        average = int(sumValues / len(sentenceValue))
        
        # Generate summary based on sentence values
        summary = ''
        for sentence in sentences:
            if sentenceValue.get(sentence, 0) > (1.5 * average):
                summary += " " + sentence
        
        # Append summary to results
        results_summarization.append(summary)
    else:
        # If the abstract is short, use it as is
        results_summarization.append(abstract)


[nltk_data] Error loading stopwords: <urlopen error [WinError 10054]
[nltk_data]     An existing connection was forcibly closed by the
[nltk_data]     remote host>
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sdany\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Summarizing: 100%|███████████████████████████████████████████████████████████████| 1615/1615 [00:01<00:00, 1245.42it/s]


In [10]:
df['Summarization']=results_summarization
df['Summarization']

0       abstract:to address air force (af) spacecraft ...
1       leveraging the successful proof-of-concept hyb...
2       in this proposed effort, rsa will develop a de...
3       the growing international interest in missions...
4       our adversaries have demonstrated cruise missi...
                              ...                        
1610    pretalen proposes a broadcast antenna technolo...
1611    since the first navstar gps satellite was laun...
1612    since the first navstar gps satellite was laun...
1613    current navy reentry flight test bodies use a ...
1614    current navy reentry flight test bodies use a ...
Name: Summarization, Length: 1615, dtype: object

#### Classifying

In [ ]:
for abstract in tqdm(df['Research Keywords'], desc="Classifying"):
    try:
        result = classifier(abstract, candidate_labels)
        results_classification.append(result)
        max_score_index = result['scores'].index(max(result['scores']))
        predicted_labels.append(result['labels'][max_score_index])
        print(len(results_classification),len(predicted_labels),'\n')
    
    except Exception as e:
        print(f"Error processing abstract: {e}")
        results_classification.append(None)
        predicted_labels.append(None)
        print(len(results_classification),len(predicted_labels),'\n')

Classifying:   0%|                                                                  | 1/1615 [00:03<1:27:56,  3.27s/it]

1 1 



Classifying:   0%|                                                                  | 2/1615 [00:06<1:22:45,  3.08s/it]

2 2 



Classifying:   0%|                                                                  | 3/1615 [00:08<1:16:19,  2.84s/it]

...

130 130 



Classifying:   8%|█████▏                                                          | 131/1615 [09:05<1:47:06,  4.33s/it]

131 131 



Classifying:   8%|█████▏                                                          | 132/1615 [09:09<1:45:06,  4.25s/it]

132 132 



In [ ]:
for label in candidate_labels:
    df[change_labels(label)] = [result['scores'][result['labels'].index(label)] if result and label in result['labels'] else 0 for result in results_classification]

In [ ]:
df['Predicted Label'] = [change_labels(label) for label in predicted_labels] 


### Saving on output path

In [ ]:
output_file_path = f'C:/Users/sdany/Desktop/pilot project/output_50jup.xlsx'
df.to_excel(output_file_path, index=False)

print(f"Output file path: {output_file_path}")

### Open Excel file

In [ ]:
if os.name == 'nt':
    os.startfile(output_file_path)
elif os.name == 'posix':
    subprocess.call(['open', output_file_path])
else:
    subprocess.call(['xdg-open', output_file_path])

In [ ]:
if os.name == 'nt':
    os.startfile(excel_file_path)
elif os.name == 'posix':
    subprocess.call(['open', excel_file_path])
else:
    subprocess.call(['xdg-open', excel_file_path])